In [4]:
!pip3 install transformers pandas
!pip3 install torch --extra-index-url https://download.pytorch.org/whl/cu113


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\ericl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu113



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: C:\Users\ericl\AppData\Local\Programs\Python\Python310\python.exe -m pip install --upgrade pip


In [23]:
import pandas as pd
from transformers import pipeline
import re

In [29]:
df = pd.read_csv("data/stock_tweets.csv")

# Data Preprocessing to remove noise
def clean_tweet(text):
    # Remove mentions
    text = re.sub(r'@\w+', '', text)
    # Remove hashtags
    text = re.sub(r'#\w+', '', text)
    # Remove URLs
    text = re.sub(r'http\S+|www\S+', '', text)
    # Remove emojis using Unicode ranges
    text = re.sub(r'[\U0001F600-\U0001F64F'
                  r'\U0001F300-\U0001F5FF'
                  r'\U0001F680-\U0001F6FF'
                  r'\U0001F700-\U0001F77F'
                  r'\U0001F780-\U0001F7FF'
                  r'\U0001F800-\U0001F8FF'
                  r'\U0001F900-\U0001F9FF'
                  r'\U0001FA00-\U0001FA6F'
                  r'\U0001FA70-\U0001FAFF'
                  r'\U00002702-\U000027B0'
                  r'\U000024C2-\U0001F251]', '', text)
    return text.strip()


# Apply the function to the "tweet" column
df['Tweet'] = df['Tweet'].apply(clean_tweet)

df.head()
# #For testing purposes
df.to_csv("new_tweets.csv", index=False)

In [30]:
sentiment_pipeline = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english", batch_size=16, device=0)

In [33]:
def analyze_sentiment(tweet):
    result = sentiment_pipeline(tweet[:512])
    return result[0]['label'], result[0]['score']

In [34]:
df[['Sentiment', 'Confidence']] = df['Tweet'].apply(lambda x: pd.Series(analyze_sentiment(x)))

KeyboardInterrupt: 

In [28]:
output_data = df[['Tweet', 'Sentiment', 'Confidence']]
output_file_path = "sentiment_analysis_results.csv"
output_data.to_csv(output_file_path, index=False)